In [29]:
import os
import pandas as pd
import csv
import json

json_mix1 = "/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/clip_experiments/mix_1_actionswap_per_background_class.json"
json_mix2 = "/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/clip_experiments/mix_2_actionswap_per_background_class.json"
json_mix3 = "/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/clip_experiments/mix_3_actionswap_per_background_class.json"
csv_path = "/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/clip_experiments/per_background_class_label.csv"

with open(json_mix1, "r") as f:
    data1 = json.load(f)

with open(json_mix2, "r") as f:
    data2 = json.load(f)

with open(json_mix3, "r") as f:
    data3 = json.load(f)

df = pd.read_csv("/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/kinetics_400_labels.csv")
classes = [name for name in df["name"]]

rows = []

datasets = [data1, data2, data3]

for label in classes:
    pred_human = sum(d.get(label, {}).get("pred_human", 0) for d in datasets)
    pred_background = sum(d.get(label, {}).get("pred_background", 0) for d in datasets)
    total = sum(d.get(label, {}).get("total", 0) for d in datasets)
    rows.append({
        "background_class":label,
        "pred_human":pred_human,
        "pred_background":pred_background,
        "total":total,
        "human_acc":pred_human/total if total else 0.0,
        "background_error":pred_background/total if total else 0.0
    })

pd.DataFrame(rows).to_csv(csv_path, index=False)


In [30]:
clip_df = pd.read_csv(csv_path)
df_sorted = clip_df.sort_values(by="background_error", ascending=False)
df_sorted.to_csv("/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/clip_experiments/per_background_class_label.csv", index=False)